---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('../materials/A4_graphs','rb'))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
for G in P1_Graphs:
    print(nx.average_clustering(G), nx.average_shortest_path_length(G))

0.03167539146454044 4.099161161161161
0.5642419635919628 5.089871871871872
0.4018222222222227 9.378702269692925
0.03780379975223251 3.1048046283934134
0.0033037037037037037 5.0785509568313305


In [4]:
def graph_identification():
    
    # Return values
    # 1 and 4 have low clustering coef and low average distance - PA
    # 2 and 3 have high clustering coef and low average distance - SW_L
    # 5 has very low clustering coef and low average distance - SW_H
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('../materials/email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
from sklearn.ensemble import RandomForestClassifier

def salary_predictions():
    
    # Initialize the dataframe, using the nodes as the index
    df = pd.DataFrame(index=G.nodes())
    
    # Extract node atributes from the graph
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

    # Derive node-level features
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Degree'] = pd.Series(G.degree())
    df['DegreeCentrality'] = pd.Series(nx.degree_centrality(G))
    df['BetweenessCentrality'] = pd.Series(nx.betweenness_centrality(G))
    df['ClosenessCentrality'] = pd.Series(nx.closeness_centrality(G))
    df['PageRank'] = pd.Series(nx.pagerank(G))

    # Define train and test sets
    f = ['Clustering', 'Degree', 'DegreeCentrality', 'BetweenessCentrality', 'ClosenessCentrality', 'PageRank']
    X_train = df[df.ManagementSalary.notnull()][f]
    y_train = df[df.ManagementSalary.notnull()]['ManagementSalary']
    X_test = df[df.ManagementSalary.isnull()][f]

    # Train classifier
    clf = RandomForestClassifier(random_state = 0, n_estimators = 100, max_depth = 10).fit(X_train.as_matrix(), 
                                                                                           y_train.as_matrix())
    
    # Predict
    pred = [scores[1] for scores in clf.predict_proba(X_test.as_matrix())]
    
    # Return as Pandas Series
    return pd.Series(data = pred,  index = X_test.index)

In [7]:
len(salary_predictions())

252

**Achieved AUC:** 0.945620589456206

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [8]:
future_connections = pd.read_csv('../materials/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [9]:
def new_connections_predictions():
    
    # Derive edge-level features
    future_connections['Preferential Attachment'] = [list(nx.preferential_attachment(G, [edge]))[0][2] for edge in future_connections.index]
    future_connections['Common Neighbors'] = [len(list(nx.common_neighbors(G, edge[0], edge[1]))) for edge in future_connections.index]
    future_connections['Jaccard Coefficient'] = [list(nx.jaccard_coefficient(G, [edge]))[0][2] for edge in future_connections.index]
    future_connections['Resource Allocation Index'] = [list(nx.resource_allocation_index(G, [edge]))[0][2] for edge in future_connections.index]
    future_connections['Adamic Adar Index'] = [list(nx.adamic_adar_index(G, [edge]))[0][2] for edge in future_connections.index]
    future_connections['CN Soundarajan Hopcroft'] = [list(nx.cn_soundarajan_hopcroft(G, ebunch=[edge], community='Department'))[0][2] for edge in future_connections.index]
    future_connections['RA Soundarajan Hopcroft'] =  [list(nx.ra_index_soundarajan_hopcroft(G, ebunch=[edge], community='Department'))[0][2] for edge in future_connections.index]
    
    # Define train and test sets
    f = ['Preferential Attachment','Common Neighbors', 'Jaccard Coefficient', 'Resource Allocation Index', 
                'Adamic Adar Index','CN Soundarajan Hopcroft', 'RA Soundarajan Hopcroft']
    X_train = future_connections[future_connections['Future Connection'].notnull()][f]
    y_train = future_connections[future_connections['Future Connection'].notnull()]['Future Connection']
    X_test = future_connections[future_connections['Future Connection'].isnull()][f]

     # Train classifier
    clf = RandomForestClassifier(random_state = 0, n_estimators = 100, max_depth = 10).fit(X_train.as_matrix(), 
                                                                                           y_train.as_matrix())
    
    # Predict
    pred = [scores[1] for scores in clf.predict_proba(X_test.as_matrix())]
    
    # Return as Pandas Series
    return pd.Series(data = pred,  index = X_test.index)

In [10]:
len(new_connections_predictions())

122112

**Achieved AUC:** 0.9140030198006269